In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
org_data = pd.read_csv("rule_based_data.csv",index_col=False)
if "Unnamed: 0" in org_data.columns:
    org_data = org_data.drop(columns=["Unnamed: 0"])
# crop_data.drop(['WATER_SOURCE'],axis=1)
# crop_data = crop_data[30:60]

In [3]:
org_data

,CROPS,SOIL,SOIL_PH,CROP_DURATION,TEMP,WATER_SOURCE,WATER_REQUIRED,RELATIVE_HUMIDITY
0,rice,Alluvia or loamy and clayey soil,6.8,150,31.58,"irrigated,rainfall",2307,76.87
1,rice,Alluvia or loamy and clayey soil,5.8,150,23.58,"irrigated,rainfall",2365,70.56
2,rice,Alluvia or loamy and clayey soil,5.0,150,23.02,"irrigated,rainfall",2458,79.89
3,rice,Alluvia or loamy and clayey soil,6.4,150,24.76,"irrigated,rainfall",1922,63.53
4,rice,Alluvia or loamy and clayey soil,6.8,150,30.13,"irrigated,rainfall",2226,78.30
...,...,...,...,...,...,...,...,...
895,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,6.5,319,25.09,irrigated,1967,65.69
896,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,7.4,320,23.36,irrigated,2025,76.05
897,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,7.3,320,21.93,irrigated,2240,66.09
898,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,7.4,327,25.18,irrigated,1603,69.41


In [4]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

synthetic_df=pd.DataFrame()
for i in range(30):
    if i>=29:
        crop_data = org_data[30*((i)%30):900]
    else:
        crop_data = org_data[30*((i)%30):30*((i+1)%30)]
    # Extract numerical columns
    numerical_columns = ["SOIL_PH", "CROP_DURATION", "TEMP", "WATER_REQUIRED", "RELATIVE_HUMIDITY"]
    categorical_columns = ["CROPS", "WATER_SOURCE", "SOIL"]

    # Create a MinMaxScaler
    scaler = MinMaxScaler()
    # Normalize only numerical columns
    crop_data[numerical_columns] = scaler.fit_transform(crop_data[numerical_columns])

    # Perform one-hot encoding for categorical columns
    encoder = OneHotEncoder(sparse=False, drop='first')
    categorical_encoded = encoder.fit_transform(crop_data[categorical_columns])
    categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names(categorical_columns))

    # Concatenate the encoded categorical columns with the normalized numerical columns
    normalized_data = pd.concat([crop_data[numerical_columns], categorical_encoded_df], axis=1)
    # GAN parameters
    latent_dim = 100
    num_samples = len(normalized_data)
    epochs = 1000
    batch_size = 64
    # Generator model
    generator = keras.Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(256, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dense(len(normalized_data.columns), activation="sigmoid")  # Output layer with same dimensions as input
    ])
    # Discriminator model
    discriminator = keras.Sequential([
        layers.Input(shape=(len(normalized_data.columns),)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Binary classification output
    ])
    # GAN model (combining generator and discriminator)
    discriminator.compile(loss="binary_crossentropy", optimizer="adam")
    discriminator.trainable = False
    gan_input = keras.Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = keras.Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer="adam")

    # Training loop
    for epoch in range(epochs):
        noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
        generated_data = generator.predict(noise)

        real_data_indices = np.random.choice(len(crop_data), batch_size)
        real_data = crop_data.drop(columns=["CROPS","WATER_SOURCE","SOIL"]).iloc[real_data_indices].values.astype(np.float32)

        real_data_label = np.ones((batch_size, 1), dtype=np.float32)
        fake_data_label = np.zeros((batch_size, 1), dtype=np.float32)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_data, real_data_label)
        d_loss_fake = discriminator.train_on_batch(generated_data, fake_data_label)

        # Train generator (via GAN model)
        noise = np.random.normal(0, 1, size=(batch_size, latent_dim)).astype(np.float32)
        g_loss = gan.train_on_batch(noise, real_data_label)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: D Loss Real: {d_loss_real:.4f}, D Loss Fake: {d_loss_fake:.4f}, G Loss: {g_loss:.4f}")
    
    # Generate synthetic crop data
    num_synthetic_samples = 500
    noise = np.random.normal(0, 1, size=(num_synthetic_samples, latent_dim))
    synthetic_data = generator.predict(noise)
    synthetic_data_denormalized = (synthetic_data * (scaler.data_max_ - scaler.data_min_)) + scaler.data_min_
    
    for col in normalized_data.columns:
        col_min = org_data[col].min()
        col_max = org_data[col].max()
        synthetic_data_denormalized[:, normalized_data.columns.get_loc(col)] = np.clip(
            synthetic_data_denormalized[:, normalized_data.columns.get_loc(col)], col_min, col_max
        )
    
    decimal_places = {"SOIL_PH": 1, "CROP_DURATION": 0, "TEMP": 2, "WATER_REQUIRED": 0, "RELATIVE_HUMIDITY": 2}
    synthetic_data_denormalized_rounded = synthetic_data_denormalized.copy()
    for column, places in decimal_places.items():
        synthetic_data_denormalized_rounded[:, normalized_data.columns.get_loc(column)] = np.round(synthetic_data_denormalized[:, normalized_data.columns.get_loc(column)], places)
    

    epoch_synthetic_df = pd.DataFrame(synthetic_data_denormalized_rounded, columns=normalized_data.columns)
    epoch_synthetic_df.insert(0, 'CROPS', org_data["CROPS"][30*((i)%30)])
    epoch_synthetic_df.insert(1, 'SOIL', org_data["SOIL"][30*((i)%30)])
    epoch_synthetic_df.insert(4, 'WATERSOURCE', org_data["WATER_SOURCE"][30*((i)%30)])
    synthetic_df = synthetic_df.append(epoch_synthetic_df, ignore_index=True)
    

c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


2/2 [==============================] - 0s 3ms/step
Epoch 0: D Loss Real: 0.7019, D Loss Fake: 0.7700, G Loss: 0.6457
2/2 [==============================] - 0s 3ms/step
Epoch 100: D Loss Real: 0.0054, D Loss Fake: 0.0117, G Loss: 4.4731
2/2 [==============================] - 0s 2ms/step
Epoch 200: D Loss Real: 0.0022, D Loss Fake: 0.0134, G Loss: 4.3362
2/2 [==============================] - 0s 3ms/step
Epoch 300: D Loss Real: 0.2495, D Loss Fake: 0.1420, G Loss: 2.0491
2/2 [==============================] - 0s 3ms/step
Epoch 400: D Loss Real: 0.0132, D Loss Fake: 0.0158, G Loss: 4.1808
2/2 [==============================] - 0s 2ms/step
Epoch 500: D Loss Real: 0.0203, D Loss Fake: 0.0265, G Loss: 3.7995
2/2 [==============================] - 0s 3ms/step
Epoch 600: D Loss Real: 0.1312, D Loss Fake: 0.0608, G Loss: 2.8832
2/2 [==============================] - 0s 4ms/step
Epoch 700: D Loss Real: 0.2770, D Loss Fake: 0.5653, G Loss: 3.8373
2/2 [==============================] - 0s 3ms/step

c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 0: D Loss Real: 0.7193, D Loss Fake: 0.7787, G Loss: 0.6292
2/2 [==============================] - 0s 3ms/step
Epoch 100: D Loss Real: 0.0362, D Loss Fake: 0.0214, G Loss: 3.8583
2/2 [==============================] - 0s 3ms/step
Epoch 200: D Loss Real: 0.0006, D Loss Fake: 0.0018, G Loss: 6.3176
2/2 [==============================] - 0s 3ms/step
Epoch 300: D Loss Real: 0.0612, D Loss Fake: 0.0564, G Loss: 2.9959
2/2 [==============================] - 0s 4ms/step
Epoch 400: D Loss Real: 0.2557, D Loss Fake: 0.0158, G Loss: 4.1513
2/2 [==============================] - 0s 3ms/step
Epoch 500: D Loss Real: 0.2046, D Loss Fake: 0.0023, G Loss: 6.6301
2/2 [==============================] - 0s 2ms/step
Epoch 600: D Loss Real: 2.0440, D Loss Fake: 0.0623, G Loss: 2.8111
2/2 [==============================] - 0s 3ms/step
Epoch 700: D Loss Real: 0.2925, D Loss Fake: 0.3354, G Loss: 2.8254
2/2 [==============================] - 0s 3ms/step
Epoch 800: D Loss Real: 0.4966, D Loss Fake: 0.428

c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


...

2/2 [==============================] - 0s 17ms/step
Epoch 0: D Loss Real: 0.7380, D Loss Fake: 0.7331, G Loss: 0.6647
2/2 [==============================] - 0s 16ms/step
Epoch 100: D Loss Real: 0.0283, D Loss Fake: 0.0166, G Loss: 4.1337
2/2 [==============================] - 0s 16ms/step
Epoch 200: D Loss Real: 0.0010, D Loss Fake: 0.0005, G Loss: 7.6359
2/2 [==============================] - 0s 15ms/step
Epoch 300: D Loss Real: 0.0340, D Loss Fake: 0.0451, G Loss: 3.1641
2/2 [==============================] - 0s 19ms/step
Epoch 400: D Loss Real: 0.8108, D Loss Fake: 0.4703, G Loss: 1.3856
2/2 [==============================] - 0s 17ms/step
Epoch 500: D Loss Real: 0.0113, D Loss Fake: 0.0020, G Loss: 6.2198
2/2 [==============================] - 0s 14ms/step
Epoch 600: D Loss Real: 0.1230, D Loss Fake: 0.0425, G Loss: 3.3214
2/2 [==============================] - 0s 13ms/step
Epoch 700: D Loss Real: 0.7721, D Loss Fake: 0.2704, G Loss: 2.7169
2/2 [==============================] - 0s 

c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


2/2 [==============================] - 0s 21ms/step
Epoch 0: D Loss Real: 0.7035, D Loss Fake: 0.7941, G Loss: 0.6126
2/2 [==============================] - 0s 15ms/step
Epoch 100: D Loss Real: 0.2099, D Loss Fake: 1.1485, G Loss: 1.5019
2/2 [==============================] - 0s 15ms/step
Epoch 200: D Loss Real: 0.0510, D Loss Fake: 0.0167, G Loss: 4.1281
2/2 [==============================] - 0s 15ms/step
Epoch 300: D Loss Real: 0.1122, D Loss Fake: 0.0524, G Loss: 2.9834
2/2 [==============================] - 0s 15ms/step
Epoch 400: D Loss Real: 0.1655, D Loss Fake: 0.0639, G Loss: 3.2639
2/2 [==============================] - 0s 16ms/step
Epoch 500: D Loss Real: 1.4428, D Loss Fake: 0.3050, G Loss: 1.5429
2/2 [==============================] - 0s 16ms/step
Epoch 600: D Loss Real: 0.3028, D Loss Fake: 0.1980, G Loss: 2.6530
2/2 [==============================] - 0s 16ms/step
Epoch 700: D Loss Real: 0.2411, D Loss Fake: 0.2300, G Loss: 1.8493
2/2 [==============================] - 0s 

c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


2/2 [==============================] - 0s 17ms/step
Epoch 0: D Loss Real: 0.6893, D Loss Fake: 0.8084, G Loss: 0.6046
2/2 [==============================] - 0s 16ms/step
Epoch 100: D Loss Real: 0.0033, D Loss Fake: 0.0043, G Loss: 5.4720
2/2 [==============================] - 0s 14ms/step
Epoch 200: D Loss Real: 0.2926, D Loss Fake: 0.1429, G Loss: 2.0669
2/2 [==============================] - 0s 17ms/step
Epoch 300: D Loss Real: 0.2364, D Loss Fake: 0.1260, G Loss: 2.3215
2/2 [==============================] - 0s 16ms/step
Epoch 400: D Loss Real: 0.0224, D Loss Fake: 0.2292, G Loss: 2.1680
2/2 [==============================] - 0s 15ms/step
Epoch 500: D Loss Real: 0.0195, D Loss Fake: 0.0034, G Loss: 5.6998
2/2 [==============================] - 0s 18ms/step
Epoch 600: D Loss Real: 0.2375, D Loss Fake: 0.4991, G Loss: 3.3638
2/2 [==============================] - 0s 16ms/step
Epoch 700: D Loss Real: 0.6032, D Loss Fake: 0.3917, G Loss: 1.4692
2/2 [==============================] - 0s 

In [7]:
synthetic_df

,CROPS,SOIL,SOIL_PH,CROP_DURATION,WATERSOURCE,TEMP,WATER_REQUIRED,RELATIVE_HUMIDITY
0,rice,Alluvia or loamy and clayey soil,6.5,150.0,"irrigated,rainfall",25.63,1804.0,68.89
1,rice,Alluvia or loamy and clayey soil,5.8,150.0,"irrigated,rainfall",24.94,1846.0,68.23
2,rice,Alluvia or loamy and clayey soil,6.0,150.0,"irrigated,rainfall",24.81,1841.0,67.05
3,rice,Alluvia or loamy and clayey soil,6.1,150.0,"irrigated,rainfall",23.09,1332.0,72.31
4,rice,Alluvia or loamy and clayey soil,5.0,150.0,"irrigated,rainfall",20.07,918.0,79.89
...,...,...,...,...,...,...,...,...
14995,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,1.0,327.0,irrigated,1.00,2192.0,1.00
14996,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,1.0,301.0,irrigated,1.00,2466.0,1.00
14997,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,1.0,328.0,irrigated,1.00,2246.0,1.00
14998,sugarcane,Clayey Loamy Soil/ Black Cotton Soil/ Red Loam...,1.0,311.0,irrigated,1.00,2109.0,1.00


In [8]:
# Save synthetic data to CSV
synthetic_df.to_csv("synthetic_crop_data1.csv")

In [ ]:

synthetic_df.to_csv("crop_data1.csv")